# Data Pipeline
For this project determining which strategies to use determines the sucess of the trading bot.<br>
In the context of this project we follow this structure in 3 steps found in these folders in `src/`<br>
`backtesting/` -> `data/` -> `models/`<br>
1. `backtesting/` - Here we try out any choice of strategies we want to try using `backtrader` to find a viable technical indicator based approach to ensure that historically our selection would turn a profit.
2. `data/` - Here we then implement those technical indicators onto a `DataFrame` from historical data on a stock of our choice. This also includes the signals to indicate that we should buy or sell, this makes the `DataFrame` contain all the features and labels necessary to train model on.
3. `models/` - Here we train the machine learning models on the modified `DataFrame`s and validate that they can then accurately predict the buy and sell signals based on technical indicators<br>
Given this pipeline we can ensure the validity of the technical indicators and allow machine learning models to train on them. 

# Backtesting
We first test the strategy on historical data using technicals of our choice using `backtrader`, once we decide what technicals indicators we would like to use we then apply them in the `process_data()` method found in `src/data/data_processing.py`. After we define the modifications to the `DataFrame` then we can start training the ML models

In [3]:
import sys
import os
import sys
sys.path.insert(0, '../src')

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from data import data_processing as dp # to get modified DataFrames with Technicals

# Get DataFrame
In `src/data/data_processing.py` we can load a `DataFrame` that contains our own technical indicators using the `get_df` method and a spcecified ticker. We can modify the strategy and technical indicators in the `process_data` method to adjust to new strategies. This method will also add the signals to the df to allow ML models to train on. 

In [6]:
df = dp.get_df("AAPL")
df

[*********************100%***********************]  1 of 1 completed


Price,close,high,low,open,volume,RSI,BB_LOWER,BB_UPPER,MACD,SIGNAL,SMA(10),SMA(30),SMA_CROSS,MACD_CROSS,BB_SELL,BB_BUY,RSI_SIG
Date,,,,,,,,,,,,,,,,,
1981-01-26,0.110600,0.111028,0.110600,0.111028,24640000,53.741977,0.100326,0.122203,0.000836,0.000747,0.109228,0.107385,0,0,0,0,0
1981-01-27,0.109742,0.110600,0.109742,0.110600,23699200,52.368639,0.101259,0.119898,0.000735,0.000744,0.109742,0.107756,0,-1,0,0,0
1981-01-28,0.106313,0.106742,0.106313,0.106742,28156800,47.175715,0.101624,0.118119,0.000395,0.000674,0.109871,0.108185,0,0,0,0,0
1981-01-29,0.102455,0.102884,0.102455,0.102884,43904000,42.116051,0.100983,0.117301,-0.000161,0.000507,0.109399,0.108714,0,0,0,0,0
1981-01-30,0.096882,0.097739,0.096882,0.097739,46188800,36.093548,0.099372,0.116769,-0.001009,0.000204,0.108456,0.108985,-1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-08,197.231369,199.788014,194.425036,197.461064,50478900,42.763668,188.205137,215.594344,-2.568219,-2.447572,205.025142,203.018774,0,-1,0,0,0
2025-05-09,198.270004,200.277366,197.281295,198.739390,36453900,43.869374,189.629429,214.979991,-2.679778,-2.494013,203.951550,202.175879,0,0,0,0,0
2025-05-12,210.789993,211.270004,206.750000,210.970001,63775800,55.123511,189.912549,215.986821,-1.737897,-2.342790,204.044070,201.948392,0,1,0,0,0


In [17]:
# Add signals
df['SIGNAL'] = df.iloc[:, 12] + df.iloc[:, 13] + df.iloc[:, 14] + df.iloc[:, 15] + df.iloc[:, 16]
len(df[df['SIGNAL'] != 0])

4364

# Construct ML model
Once we have a dataframe with the technical indicators we would like to use, we can construct the ML model. We will use scikit-learn to simplify the process. Here we will scale the data, set up the pipeline, and split the data up. However most of the signals in the `DataFrame` are just "HOLD" so we need to do a lot preprocessing in regards to filling in those gaps so the ML can have more diverse labels.

# Train ML Model
We will test the model on the training data which will be a chunk of the dataframe we got from yfinance. Then once trained we can put it another chunk for validation data and adjust parameters as needed. Finally we can run the model on the test data for final analysis. 

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.under_sampling import RandomUnderSampler
import warnings

X = df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8]] # All columns except the signal (feature)
y = df.iloc[:, 9] # Just the signal column (label)

rus = RandomUnderSampler(random_state=42) 
X_res, y_res = rus.fit_resample(X, y) 

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", classification_rep)


Accuracy: 0.54

Classification Report:
               precision    recall  f1-score   support

          -2       0.62      0.37      0.47        27
          -1       0.37      0.62      0.47        16
           0       0.38      0.31      0.34        16
           1       0.30      0.43      0.35        14
           2       0.96      0.85      0.90        26

    accuracy                           0.54        99
   macro avg       0.53      0.52      0.51        99
weighted avg       0.59      0.54      0.54        99



# Export Model 
Once the model is trained we can export it using the `pickle` library or another equivalent. We can then use this in a driver where we can then feed the bot live data from the `finnhub` API and then recompute the technicals used to train the bot and allow it to decide and execute trades through the Alpaca API. 

# Test model on new data 